In [12]:
import torch
from xitorch.optimize import rootfinder
import matplotlib.pyplot as plt

# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# Move the tensors to the appropriate device
# Replace `your_tensor` with the actual tensor variable name
# your_tensor = your_tensor.to(device)

In [13]:
def sdfNorm(point, center, zero_radius, norm=2):
    # Calculate the signed distance of a point from the set
    # Replace this with your actual signed distance function implementation

    eval = torch.zeros(point.shape[0]).to(device)

    for i in range(point.shape[0]): 

        eval[i] = torch.linalg.norm(point[i,:]-center,ord=2).to(device) - zero_radius

    return eval


def empiricalCDFGen(sdf,samples,eval):
    # Calculate the empirical CDF of the signed distance function
    # This is a simple wrapper around the numpy function


    return torch.sum(sdf(samples,eval) <= 0)/samples.shape[0]


def findZeroOneCDF(cdfFunction,startPt,tol):
    # Determine where the CDF hits 0 and 1.

    zeroOne = torch.zeros(2).to(device)

    # Find the zero crossing
    zeroOne[0] = rootfinder(cdfFunction,startPt)
    complementZero = lambda x: 1-cdfFunction(x)
    zeroOne[1] = rootfinder(complementZero,startPt)

    return zeroOne

def findLevelSet(cdfFunction,zeroOne,prob):
    # Find the level set of the CDF that corresponds to the zero radius
    # This is a simple wrapper around the optimization function

    levelCDF = lambda x: cdfFunction(x) - prob
    avgStart = (zeroOne[0]+zeroOne[1])/2    

    return rootfinder(levelCDF,avgStart)


def plot2DNormLevelSet(center,zero_radius,samples):

    # Generate samples within the norm-ball
    theta = torch.linspace(0, 2*np.pi, 100).to(device)
    x = center[0] + zero_radius * torch.cos(theta)
    y = center[1] + zero_radius * torch.sin(theta)

    # Plot the norm-ball
    plt.plot(x.detach().numpy(), y.detach().numpy(), color='blue', label='Norm-Ball')

    # Plot the samples
    plt.scatter(samples[:, 0], samples[:, 1], color='red', label='Samples')

    # Set plot labels and legend
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.legend()
    # Set the aspect ratio of the plot to 'equal'
    plt.axis('equal')

    # Show the plot
    plt.show()

In [14]:
# Mean vector
mean = torch.tensor([0.0, 0.0]).to(device)

# Covariance matrix
cov = torch.tensor([[10, 0.5], [0.5, 1]]).to(device)

alpha = torch.tensor(0.9999).to(device)
epsilon = torch.tensor(0.001).to(device)
# Number of samples
num_samples = torch.ceil(-torch.log(alpha / 2) / (2 * epsilon**2)).int()

# Generate samples
m = torch.distributions.MultivariateNormal(mean, cov)
samples = m.sample((num_samples,)).to(device)

# Center of the norm-ball
center = torch.tensor([0, 0]).to(device)

# Set up function for the signed distance function

sdf = lambda samples,zero_radius: sdfNorm(samples, center, zero_radius)

empiricalCDF = lambda eval: empiricalCDFGen(sdf,samples,eval)

startPt = torch.tensor(0.5).to(device)

zeroOne = findZeroOneCDF(empiricalCDF,startPt)
levelProb = 0.99999
levelSetZeroRadius = findLevelSet(empiricalCDF,zeroOne,levelProb)

print(zeroOne)
print(levelSetZeroRadius)
print(empiricalCDF(levelSetZeroRadius))

plot2DNormLevelSet(center,levelSetZeroRadius,samples)


TypeError: numel(): argument 'input' (position 1) must be Tensor, not float